# Correlation network: all comparisons and Normalized

This analysis generates a correlation network that includes intra-datasource correlations

In [1]:
from IPython.core.display import display, HTML

display(HTML('''<style>
.container {width:98% !important;}
.dataframe th{font: bold 14px times; background: #0ea; text-align: right;}
.dataframe td{font: 14px courier; background: #fff; text-align: right;}
.output_subarea.output_text.output_stream.output_stderr {background: #fff; font-style: italic;}
</style>'''))

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
try:
    run_once
except NameError:
    run_once = False
if not run_once:
    run_once = True
    
    import time
    import logging
    reload(logging)
    FORMAT = '%(asctime)s - %(name)s - %(levelname)s - %(message)s'
    log_path = '/notebooks/logs/correlation.log'
    print "logging to %s" % log_path
    logging.basicConfig(filename=log_path,level=logging.DEBUG, format=FORMAT)
    logger = logging.getLogger()
    #logger.basicConfig(filename='/notebooks/Export Microbiome to database.log',level=logging.DEBUG)
    logger.setLevel(logging.DEBUG)
    ch = logging.StreamHandler()
    ch.setLevel(logging.DEBUG)

    # create formatter
    formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

    # add formatter to ch
    ch.setFormatter(formatter)

    # add ch to logger
    logger.addHandler(ch)

logging to /notebooks/logs/correlation.log


In [4]:
%matplotlib inline
import pandas, pandas.io
import re
import seaborn as sns
import math
import scipy, scipy.stats
import matplotlib.pyplot as plt
import numpy as np
import string
import os, os.path
#from p100.database.redshift import Redshift
from p100.database import Database
logging.getLogger('boto').setLevel(logging.INFO)
logging.getLogger('p100').setLevel(logging.INFO)

In [5]:
import correlationsnodb.datasource as datasource

/usr/local/lib/python2.7/dist-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: Error in dyn.load(file, DLLpath = DLLpath, ...) : 
  unable to load shared object '/usr/lib/R/library/stats/libs/stats.so':
  libRlapack.so: cannot open shared object file: No such file or directory

  warnings.warn(x, RRuntimeWarning)
/usr/local/lib/python2.7/dist-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: During startup - 
  warnings.warn(x, RRuntimeWarning)
/usr/local/lib/python2.7/dist-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: Warning message:

  warnings.warn(x, RRuntimeWarning)
/usr/local/lib/python2.7/dist-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: package ‘stats’ in options("defaultPackages") was not found 

  warnings.warn(x, RRuntimeWarning)
2016-12-14 23:18:18,512 - root - INFO - Generating grammar tables from /usr/lib/python2.7/lib2to3/Grammar.txt
2016-12-14 23:18:18,537 - root - INFO - Generating grammar tables from /usr/lib/python2.7/li

In [6]:
import correlationsnodb.analysis as analysis

In [ ]:
DS_ID_MAP= 'data/ds_id_map.pkl'                                                                                                                                                                                                             
PART_DF = 'data/participant_data.pkl' 
DATA_DIR = 'data'

In [7]:

import datetime
import itertools
def generate_correlations(args):
    rnd, entropy = args
    dsf = datasource.DataSourceFactory(ds_id_map=DS_ID_MAP, part_df=PART_DF, data_dir=DATA_DIR)
    logging.getLogger('p100.utils.correlations').setLevel(logging.DEBUG)
    # get all pairwise datasources
    bc = dsf.get_all_comparisons()
    analy = analysis.Analysis(ds_id_map=DS_ID_MAP, part_df=PART_DF)

    for c1, c2 in bc:
        try:
            analy.Correlate( c1, c2, mean=False, mean_age_sex=True, tests=[analy.spearman], cutoff=1.01)
        except:
            logging.exception("Error correlating %s and %s for round %s" % ( str(c1), str(c2), rnd) )
            raise
        logging.debug("Completed comparisong %s and %s for round %s" % ( str(c1), str(c2), rnd))
    logging.info( "Correlations %s round(%s) entropy(%0.1f)" % (datetime.date.isoformat(datetime.datetime.now()),str(rnd), entropy) )
    return analy

In [8]:
analysis = generate_correlations(('mean', 0.75))

2016-12-14 23:18:21,497 - p100.utils.correlations.datasource - DEBUG - Initializing Datasource[-1]: None, None
2016-12-14 23:18:21,509 - p100.utils.correlations.datasource - DEBUG - Initializing Datasource[-1]: METAB, None
2016-12-14 23:18:21,523 - p100.utils.correlations.datasource - DEBUG - Initializing Datasource[-1]: None, None
2016-12-14 23:18:21,535 - p100.utils.correlations.datasource - DEBUG - Initializing Datasource[-1]: PROTE, None
2016-12-14 23:18:21,547 - p100.utils.correlations.datasource - DEBUG - Initializing Datasource[-1]: None, None
2016-12-14 23:18:21,560 - p100.utils.correlations.datasource - DEBUG - Initializing Datasource[-1]: METAB, None
2016-12-14 23:18:21,572 - p100.utils.correlations.datasource - DEBUG - Initializing Datasource[-1]: None, None
2016-12-14 23:18:21,585 - p100.utils.correlations.datasource - DEBUG - Initializing Datasource[-1]: PROTE, None
2016-12-14 23:18:21,598 - p100.utils.correlations.datasource - DEBUG - Initializing Datasource[-1]: None, No

In [9]:
r = analysis.GetResult()
r.head()

,ds_id_1,ds_id_2,variable_id_1,variable_id_2,test,coefficient,pval,pval_adj
0,6018,6019,694,1,SPEARMAN,-0.002016,0.985840,0.998707
1,6018,6019,694,2,SPEARMAN,-0.067909,0.549487,0.952488
2,6018,6019,694,3,SPEARMAN,-0.337318,0.002215,0.397666
3,6018,6019,694,5,SPEARMAN,-0.064182,0.571659,0.956880
4,6018,6019,694,6,SPEARMAN,0.050281,0.657815,0.967577


In [10]:
r = analysis.GetResult(annotated=True)
r.head()

2016-12-15 04:46:45,350 - p100.utils.correlations.analysis - DEBUG - annotating
2016-12-15 04:46:45,395 - p100.utils.correlations.datasource - DEBUG - Initializing Datasource[-1]: None, None
2016-12-15 04:46:45,409 - p100.utils.correlations.datasource - DEBUG - Initializing Datasource[-1]: METAB, None
2016-12-15 04:46:47,714 - p100.utils.correlations.analysis - DEBUG - 6018: annotated
2016-12-15 04:46:47,730 - p100.utils.correlations.datasource - DEBUG - Initializing Datasource[-1]: None, None
2016-12-15 04:46:47,743 - p100.utils.correlations.datasource - DEBUG - Initializing Datasource[-1]: PROTE, None
2016-12-15 04:46:48,510 - p100.utils.correlations.analysis - DEBUG - 6019: annotated
2016-12-15 04:46:48,527 - p100.utils.correlations.datasource - DEBUG - Initializing Datasource[-1]: None, None
2016-12-15 04:46:48,539 - p100.utils.correlations.datasource - DEBUG - Initializing Datasource[-1]: METAB, None
2016-12-15 04:46:49,595 - p100.utils.correlations.analysis - DEBUG - 6032: annota

,ds_id_1,ds_id_2,variable_id_1,variable_id_2,test,coefficient,pval,pval_adj,annotations_1,annotations_2
0,6018,6019,694,1,SPEARMAN,-0.002016,0.985840,0.998707,"METAB.None.xenobiotics.chemical.1,2,3-benzenet...",PROTE.None.Inflammation.IL_8.None
1,6018,6019,694,2,SPEARMAN,-0.067909,0.549487,0.952488,"METAB.None.xenobiotics.chemical.1,2,3-benzenet...",PROTE.None.Inflammation.VEGF_A.None
2,6018,6019,694,3,SPEARMAN,-0.337318,0.002215,0.397666,"METAB.None.xenobiotics.chemical.1,2,3-benzenet...",PROTE.None.Inflammation.BDNF.None
3,6018,6019,694,5,SPEARMAN,-0.064182,0.571659,0.956880,"METAB.None.xenobiotics.chemical.1,2,3-benzenet...",PROTE.None.Inflammation.MCP_3.None
4,6018,6019,694,6,SPEARMAN,0.050281,0.657815,0.967577,"METAB.None.xenobiotics.chemical.1,2,3-benzenet...",PROTE.None.Inflammation.hGDNF.None
